In [1]:
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import pyarrow as pa
from urllib.parse import quote_plus
from pandas.api.types import is_string_dtype
import gc
import warnings

import config
import config_media_costs
from normalize_funcs import normalize_columns_types, append_custom_columns
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table, removeRowsFromDB
from create_dicts import get_cleaning_dict, get_media_discounts
from create_dicts_adex import get_adex_dicts

db_name = config.db_name

In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

In [3]:
# создаем функцию, которая вернет номер месяца от начальной даты отсчета
def get_mon_num_from_date(curr_date):
    curr_date = datetime.strptime(str(curr_date), '%Y-%m-%d')
    # с помощью метода relativedelta получаем кол-во лет и месяцев от даты начала до текущей даты
    relative_date = relativedelta(curr_date, config_media_costs.start_of_the_time)
    # теперь нужно получить номер текущего месяца от даты начала
    months_count = relative_date.years * 12 + relative_date.months
    
    return months_count

In [4]:
start_date = '2020-01-01'
mon_num = get_mon_num_from_date(start_date)
print(mon_num)

360


In [8]:
# создаем фильтр, который будем применять во воложенном запросе для фильтрации ВСЕХ ТАБЛИЦ по ВСЕМ ИСТОЧНИКАМ
# этот фильтр используем в ТВ, Радио, ООН, Пресса
# Префикс для фильтруемой таблицы задан с запасом = t10
main_filter_str = f't10.sid2 in (2272,2277)'

In [ ]:
first_row_query_dict = {
    'tv': 't1.vid, t1.cid, t1.distr, t1.mon, t1.from_mon, t1.estat, t1.cnd_cost_rub,  t1.vol, t1.cnt'
}

In [9]:
media_type = 'tv'
query = f"""select 
distinct t3.sbid
from {media_type}_Ad_month t1 left join {media_type}_Appendix t10 
on t1.vid=t10.vid
left join tv_Appendix t3
on t1.vid=t3.vid
where t1.mon>={str(mon_num)}  and 
({main_filter_str})"""
# отправляем запрос в БД Медиа инвестиции

df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [10]:
df.shape

(852, 1)

In [12]:
file_path = r'C:\Users\o.bogomolov\Downloads\Telegram'
file_name = 'subbrand_id.xlsx'
df.to_excel(os.path.join(file_path, file_name))

In [ ]:
import os
import pandas as pd

file_path = r'C:\Users\o.bogomolov\Downloads\Telegram'
file_name = 'simple_02_01_2023.xlsx'
df.to_excel(os.path.join(file_path, file_name))

In [3]:
import os
import gc
import sys
import re
import json
from datetime import datetime, date, timedelta, time
import time
from dateutil.relativedelta import relativedelta
import warnings
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import pyodbc
import turbodbc
from turbodbc import connect

from IPython.display import JSON
from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

In [4]:
# добавляем библиотеки для работы с ТВ индексом

%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import time
import pandas as pd
import numpy as np
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 314
Всего найдено записей: 314

Запрошены записи: 0 - 314
Всего найдено записей: 314



In [5]:
# Статистики для отчета Buying
nat_tv_bying_statistics = ['SalesRtgPer', 'StandSalesRtgPer', 'ConsolidatedCostRUB', 'Quantity']

In [6]:

# список срезов, по которым будет разбивка отчета
slices = config.nat_tv_buying_slices
# список метрик для отчета Simple
statistics = nat_tv_bying_statistics
# Здесь указаны логические условия ad_filter 
# Они применяются для получения статистики в отчетах Simple и Buying# условия фильтрации для запроса 
ad_filter = config.nat_tv_ad_filter
# Опции для расчета - вся рф и тд.
options = config.nat_tv_options

In [7]:
weekday_filter = None
# Задаем тип дня
daytype_filter = None
# Задаем временной интервал
time_filter = None
# Задаем ЦА
basedemo_filter = None
# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None
# Задаем место просмотра
location_filter=None
# Задаем каналы
company_filter = None
# Указываем фильтр программ: продолжительность от 5 минут (300 секунд) 
program_filter = None
# Фильтр блоков
break_filter = None
sortings = None

In [8]:
start_date = '2023-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d').date()

end_date = '2023-01-02'
end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

print(f'start_date: {start_date} / end_date: {end_date}')

start_date: 2023-01-01 / end_date: 2023-01-02


In [9]:
 date_filter = [(str(start_date), str(start_date))]

In [10]:
# Посчитаем задания в цикле
tasks = []
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                    daytype_filter=daytype_filter, company_filter=company_filter, 
                                    location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                    targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                    break_filter=break_filter, ad_filter=ad_filter, 
                                    slices=slices, statistics=statistics, sortings=sortings, options=options)


# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_simple_task(task_json))
# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

Расчет задачи (id: 1eeead79-2aa0-4030-9bed-a95cfdcca1fa) [= = = = = = ] время расчета: 0:00:19.356098


In [11]:
df.shape

(94, 15)

In [12]:
df.head(1)

,adTypeId,regionId,adStartTime,researchDate,adPositionTypeName,adId,adSpotId,adPrimeTimeStatusName,adStandardDuration,tvCompanyId,adDistributionTypeName,SalesRtgPer,StandSalesRtgPer,Quantity,ConsolidatedCostRUB
0,23,99,17:35:38,2023-01-01,Первый,4400510,4869856617,Прайм-тайм,20,1870,Сетевой,2.323279,2.323279,1.0,546128.0


In [ ]:
df['ConsolidatedCostRUB'].sum()